In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
import numpy as np
import pandas as pd
import random
import time as t

In [ ]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [ ]:
numeric_file = '/home/jonscat/Study/diploma_data/train_num_test.csv'

In [ ]:
def create_df(filename):
    text = sc.textFile(filename)
    header_names = ['Id','F1','F2','Response']#list(text.first().split(','))[0:3] + [list(text.first().split(','))[-1]]
    header_row = text.first()
    no_header_text = text.filter(lambda row: row != header_row)
    
    object_rows = Row(*header_names)  # unpack list

    schema = no_header_text.map(lambda line: line.split(','))\
        .map(lambda l: [None if i == '' else int(i) if i == 'Id' else int(float(i)*1000) for i in l])\
        .map(lambda l: l[0:3] + [l[-1]])\
        .map(lambda l: object_rows(*l))\

    df = sqlContext.createDataFrame(schema, samplingRatio=0.002)
    return df

In [ ]:
df = create_df(numeric_file)

In [ ]:
df2 = df

In [ ]:
df2.show()
df2.printSchema()

In [ ]:
df2 = df2.withColumn('lol', sqrt(df2.Id))

In [ ]:
df2.limit(10).toPandas()

In [ ]:
df2.describe().show()

In [ ]:
rand_vect1 = np.array([random.randint(0,9999) for i in xrange(50)])
rand_vect2 = np.array([random.randint(0,9999) for i in xrange(50)])

In [ ]:
rdd1 = sc.parallelize(rand_vect1,3)
rdd2 = sc.parallelize(rand_vect2,3)

In [ ]:
rdd3 = rdd1.zip(rdd2)

In [ ]:
rdd3.take(10)

In [ ]:
header = ['f1','f2']
object_rows = Row(*header)  # unpack list
schema = rdd3.map(lambda l:map(int,l)).map(lambda l: object_rows(*l))

In [ ]:
df1 = sqlContext.createDataFrame(schema)
df1.show()

In [ ]:
rdd4 = df1.rdd

In [ ]:
rdd4.take(10)
#rdd4.getNumPartitions()

In [ ]:
rand_vect3 = np.array([random.randint(0,9999) for i in xrange(50)])

In [ ]:
rdd5 = sc.parallelize(rand_vect3,3)
rdd5.getNumPartitions()

In [ ]:
#rdd4 = rdd4.flatMap()
rdd4.take(10)

In [ ]:
rdd6 = rdd4.zip(rdd5)

In [132]:
def mapper(row,col_name,col_value):
    data = row.asDict()
    data[col_name] = col_value
    return Row(**data)

In [129]:
rdd6 = rdd6.map(lambda l: mapper(l[0], l[1]) )

In [130]:
rdd6.take(5)

[Row(f1=6751, f2=7643, f3=6337),
 Row(f1=4613, f2=8570, f3=1844),
 Row(f1=4585, f2=3208, f3=7678),
 Row(f1=4478, f2=5343, f3=9331),
 Row(f1=2219, f2=4075, f3=5833)]

In [ ]:
header = ['f1','f2','f3']
object_rows = Row(*header)  # unpack list
schema = rdd6.map(lambda l:map(int,l)).map(lambda l: object_rows(*l))

In [ ]:
df7 = sqlContext.createDataFrame(schema)

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree

In [ ]:
l = [1,2,3,4,5]
l[1:-1]

In [ ]:
def labelData(data):
    # label: row[end], features: row[0:end-1]
    return data.map(lambda row: LabeledPoint(row[-1], row[1:-1]))

In [ ]:
training_data, testing_data = labelData(df2.rdd).randomSplit([0.8,0.2])